In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("./data/2015_사업보고서_01_재무상태표_20200515.txt", sep = "\t", encoding = "cp949")
df = data.copy()

In [ ]:
print(df.isnull().sum())
# 결측치 처리 필요!
print(df.info())

In [ ]:
# 변수 
# 재무제표종류 종목코드 회사명 시장구분 업종 업종명 결산월 결산기준일 
# 보고서종류 통화 항목코드 항목명 당기 전기 전전기 Unnamed: 15
" ".join(df.columns)

In [ ]:
# 재무제표종류
print(df["재무제표종류"].unique())
print(df["재무제표종류"].value_counts())

In [ ]:
# 종목코드 : 한국거래소에서 관리하는 회사코드로 6자리로 구성
print(len(df["종목코드"].unique()))
print(df["종목코드"].value_counts())

In [ ]:
# 회사명 : 제출인의 회사명칭
print(len(df["회사명"].unique()))
print(df["회사명"].value_counts())

In [ ]:
# 시장구분 : 유가증권시장상장법인, 코스닥시장상장법인(Kosdaq)
# 유가증권시장상장법인 : Kospi
# 코스닥 시장은 IT(Information technology), BT(Bio technology), CT(Culture technology) 
# 기업과 벤처기업의 자금조달을 목적으로 1996년 7월 개설된 첨단 벤처기업 중심 시장입니다.
print(df["시장구분"].unique())

In [ ]:
# 업종, 업종명
# 통계청 통계분류 포털 : https://kssc.kostat.go.kr:8443/ksscNew_web/link.do?gubun=001
print(df["업종명"].value_counts())
company_name = df["업종명"].unique()
company_name_sum = df["업종명"].value_counts().sort_values()
df_company_name = pd.DataFrame({"업종명":company_name, "합계":company_name_sum})

# 업종
print(df["업종"].value_counts())
company = df["업종"].unique()
company_sum = df["업종"].value_counts().sort_values()
df_company = pd.DataFrame({"업종":company, "합계":company_sum})

# 업종과 업종명이 다른 행 추출
com_left = pd.merge(df_company_name, df_company, how = "left")
print(com_left[com_left["업종"].isnull()])

com_right = pd.merge(df_company_name, df_company, how = "right")
print(com_right[com_right["업종명"].isnull()])

# 업종 != 업종명 처리방법 회의 필요

In [ ]:
# 결산월, 결산기준일
print(df["결산월"].unique())
print(df["결산기준일"].unique())

In [ ]:
# 보고서종류
print(df["보고서종류"].unique())

In [ ]:
# 통화 : 필요 없는 변수
# 항목코드 : 계정과목코드
# ifrs_xxx : 국제 기준
# dart_xxx : 국제 기준을 수정
# entity_xxx : 신규회사

In [ ]:
df[df["전전기"].isnull()].to_csv("./data/결측치확인.csv", index = False,encoding = 'utf-8')
no = df[df["전전기"].isnull()]
no_lst = no["항목명"].unique()
no_big_classifier = []
for i in no_lst:
    if "[" not in i:
        no_big_classifier.append(i)

In [ ]:
# no_big_classifier : 

In [ ]:
# data
df["당기"] = df["당기"].str.replace(",", "")
df["전기"] = df["전기"].str.replace(",", "")
df["전전기"] = df["전전기"].str.replace(",", "")
df[["당기", "전기", "전전기"]] = df[["당기", "전기", "전전기"]].astype(np.float32)

In [ ]:
# 필요 없는 변수 제거
df.drop("통화", axis = 1, inplace = True)
df.drop("종목코드", axis = 1, inplace = True)
df.drop("Unnamed: 15", axis = 1, inplace = True)

In [ ]:
sort = list(df["항목명"].unique())

dic = {i:["-"]*1801 for i in sort}

In [ ]:
a = df[df["회사명"] == "3S"][["항목명", "당기"]].T
a.rename(columns = a.iloc[0,:], inplace = True)
a.drop(index = "항목명", inplace = True)

In [ ]:
a
idx = 0
for i in range(len(a.columns)):
    dic[a.columns[i]][0] = a.iloc[0, i]

In [ ]:
lst = []
for i in sort:
    if dic[i][0] != "-":
        lst.append(i)

In [ ]:
lst.append(0)
lst.append(0)
pd.DataFrame({"3S":a.columns, "lst":lst})
# 금융리스부채

In [ ]:
a.drop("회사명", axis = 1, inplace=True)
a.reset_index(drop = True, inplace = True)

In [ ]:
b = pd.DataFrame({"회사명":["3S"]})

In [ ]:
pd.concat([b, a], axis = 1).to_csv("./data/3S전치행렬.csv", encoding = "euc-kr", index = False)

In [ ]:
# 회사명 
company = list(df["회사명"].unique())

col_num = []
col_num_name = []
for idx, name in enumerate(company):
    a = df[df["회사명"] == name][["항목명", "당기"]].T
    a.rename(columns = a.iloc[0,:], inplace = True)
    a.drop(index = "항목명", inplace = True)
    col_num.append(len(a.columns))
    col_num_name.append(name)
    
# 가장 많은 항목명을 가진 회사
for i in range(len(col_num)):
    if col_num[i] == max(col_num):
        print(col_num_name[i])

In [ ]:
a = df[df["회사명"] == "농우바이오"][["항목명", "당기"]].T
a.rename(columns = a.iloc[0,:], inplace = True)
a.drop(index = "항목명", inplace = True)
print(a)

In [ ]:
# '      금융리스부채'
# '      금융리스부채'(25,26, 34, 35)
df[df["회사명"] == "3S"].duplicated(["항목명"], keep = False)
df[df["회사명"] == "3S"]["항목명"][[25,26, 34, 35]]

In [ ]:
a = pd.DataFrame({"a":[1, 3, 4, 5, 1]})
a.duplicated("a", keep = False)

In [ ]:
df["항목명"].unique()

In [ ]:
df["항목명"][0]

In [ ]:
lst1_대분류 = []
for i in range(len(df["항목명"])):
    if df["항목명"][i][0] != " " and df["당기"].isnull()[i] == True:
        lst1_대분류.append(df["항목명"][i])
        
print(set(lst1_대분류))
pd.DataFrame({"대분류":lst1_대분류})["대분류"].unique()

In [ ]:
lst_중분류 = []
for i in range(len(df["항목명"])):
    if df["항목명"][i][:3] == "   " and df["당기"].isnull()[i] == True:
        lst_중분류.append(df["항목명"][i])
        
print(set(lst1))
pd.DataFrame({"대분류":lst_중분류})["대분류"].unique()

In [ ]:
df[["항목코드", "항목명"]].iloc[:40, :]

In [ ]:
# 항목코드 중복 확인
lst_entity_코드 = []
lst_entity_명 = []
lst_without_entity_코드 = []
lst_without_entity_명 = []

# 각 리스트에 코드 및 코드명 append
for idx in range(len(df)):
    if "entity" in df["항목코드"][idx]:
        lst_entity_코드.append(df["항목코드"][idx])
        lst_entity_명.append(df["항목명"][idx])
    else:
        lst_without_entity_코드.append(df["항목코드"][idx])
        lst_without_entity_명.append(df["항목명"][idx])
        
# !entity 코드 딕셔너리
dic_without_entity = {lst_without_entity_코드[idx]:[] for idx in range(len(lst_without_entity_코드))}

for i, j in zip(lst_without_entity_코드, lst_without_entity_명):
    dic_without_entity[i].append(j)
    
check_duplication = {i:len(set(j)) for i, j in dic_without_entity.items()}
check_duplication

In [ ]:
for idx in range(len(df)):
    if "NoncurrentAssetsOrDisposalGroupsClassifiedAsHeldForSaleOrAsHeldForDistributionToOwners" in df["항목코드"][idx]:
        print("항목코드 =", df["항목코드"][idx], "항목명 =", df["항목명"][idx], "idx =", idx)

In [ ]:
df[df["회사명"] == "케어젠"]